GROUP MEMBERS:

Anannay Jain [210110021]

Ashish Prasad [21D180009]

Mrudul Jambhulkar [21D070044]

Saurav Raj [21D070064]

In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install sympy

For single output

In [3]:
import itertools

def verify_reed_muller(truth_table, terms, n_vars):
    """
    Verify if the given Reed-Muller expansion satisfies the truth table.
    :param truth_table: Dictionary where keys are input binary strings and values are outputs (0 or 1).
    :param terms: List of terms in the Reed-Muller expansion.
    :param n_vars: Number of variables in the function.
    :return: True if terms match the truth table, False otherwise.
    """
    for input_bits, expected_output in truth_table.items():
        # Compute the function output for this input using the terms
        output = 0
        for term in terms:
            product = 1
            for i, bit in enumerate(term):
                if bit == '1':
                    product &= int(input_bits[i])
            output ^= product  # XOR the result
        if output != expected_output:
            return False
    return True

def find_reed_muller_expansion(truth_table):
    """
    Find the Positive Polarity Reed-Muller (PPRM) expansion for a given truth table.
    :param truth_table: Dictionary where keys are input binary strings and values are outputs (0 or 1).
    :return: List of terms in the PPRM expansion.
    """
    n_vars = len(next(iter(truth_table.keys())))  # Number of variables
    all_terms = list(itertools.product(['0', '1'], repeat=n_vars))  # All possible terms
    valid_expansion = []

    # Iterate over all subsets of terms
    for subset_size in range(len(all_terms) + 1):
        for terms_subset in itertools.combinations(all_terms, subset_size):
            terms_subset = ["".join(term) for term in terms_subset]  # Convert tuples to strings
            if verify_reed_muller(truth_table, terms_subset, n_vars):
                valid_expansion = terms_subset
                return valid_expansion  # Return the first valid expansion found

    return valid_expansion  # If no valid expansion is found

def pprm_to_expression(terms, n_vars):
    """
    Convert PPRM terms into a human-readable Boolean expression.
    :param terms: List of terms in PPRM (binary strings).
    :param n_vars: Number of input variables.
    :return: String representation of the PPRM expansion.
    """
    expression = []
    for term in terms:
        # Map binary to variable names
        variables = [f"x{i+1}" for i, bit in enumerate(term) if bit == '1']
        expression.append("".join(variables) if variables else "1")
    return " ⊕ ".join(expression)

# Example usage
truth_table = {
    '0000': 0,
    '0001': 1,
    '0010': 1,
    '0011': 0,
    '0100': 1,
    '0101': 0,
    '0110': 0,
    '0111': 1,
    '1000': 1,
    '1001': 0,
    '1010': 0,
    '1011': 1,
    '1100': 0,
    '1101': 1,
    '1110': 1,
    '1111': 0
}

# Find PPRM expansion
pprm_terms = find_reed_muller_expansion(truth_table)

# Convert to expression for readability
n_vars = len(next(iter(truth_table.keys())))
pprm_expression = pprm_to_expression(pprm_terms, n_vars)

print("PPRM Terms:", pprm_terms)
print("PPRM Expression:", pprm_expression)

PPRM Terms: ['0001', '0010', '0100', '1000']
PPRM Expression: x4 ⊕ x3 ⊕ x2 ⊕ x1


In [4]:
from qiskit import QuantumCircuit

def construct_toffoli_circuit(pprm_terms, n_vars):
    """
    Construct a quantum circuit using Toffoli gates from the PPRM expansion.
    :param pprm_terms: List of terms in PPRM expansion (binary strings).
    :param n_vars: Number of input variables.
    :return: QuantumCircuit object.
    """
    # Create a quantum circuit with n_vars input qubits and 1 output qubit
    qc = QuantumCircuit(n_vars + 1, name="Reversible Circuit")
    output_qubit = n_vars  # The last qubit is the output qubit

    # Iterate through PPRM terms to construct the circuit
    for term in pprm_terms:
        controls = [i for i, bit in enumerate(term) if bit == '1']
        if len(controls) == 0:  # Constant 1 term (acts as a NOT gate on the output qubit)
            qc.x(output_qubit)
        elif len(controls) == 1:  # Single control (CNOT gate)
            qc.cx(controls[0], output_qubit)
        else:  # Multi-control (Toffoli or generalized Toffoli gate)
            qc.mcx(controls, output_qubit)

    return qc

def generate_quantum_circuit_from_truth_table(truth_table):
    """
    Generate a quantum circuit from a given truth table.
    :param truth_table: Dictionary {input: output}.
    :return: QuantumCircuit object.
    """
    # Step 1: Find PPRM expansion
    pprm_terms = find_reed_muller_expansion(truth_table)

    # Step 2: Number of variables
    n_vars = len(next(iter(truth_table.keys())))

    # Step 3: Construct quantum circuit
    qc = construct_toffoli_circuit(pprm_terms, n_vars)

    return qc

# Example usage
# truth_table = {
#     '000': 1,
#     '001': 1,
#     '010': 1,
#     '011': 0,
#     '100': 1,
#     '101': 0,
#     '110': 1,
#     '111': 1
# }
# Example usage
truth_table = {
    '0000': 0,
    '0001': 1,
    '0010': 1,
    '0011': 0,
    '0100': 1,
    '0101': 1,
    '0110': 0,
    '0111': 1,
    '1000': 0,
    '1001': 0,
    '1010': 0,
    '1011': 1,
    '1100': 0,
    '1101': 1,
    '1110': 1,
    '1111': 0
}

# Find PPRM expansion
pprm_terms = find_reed_muller_expansion(truth_table)

# Convert to expression for readability
n_vars = len(next(iter(truth_table.keys())))
pprm_expression = pprm_to_expression(pprm_terms, n_vars)

print("PPRM Terms:", pprm_terms)
print("PPRM Expression:", pprm_expression)


# Generate the quantum circuit
quantum_circuit = generate_quantum_circuit_from_truth_table(truth_table)

# Display the circuit
print("Generated Quantum Circuit:")
print(quantum_circuit.draw('text'))

PPRM Terms: ['0001', '0010', '0100', '0101', '0111', '1001', '1010', '1011', '1100', '1110']
PPRM Expression: x4 ⊕ x3 ⊕ x2 ⊕ x2x4 ⊕ x2x3x4 ⊕ x1x4 ⊕ x1x3 ⊕ x1x3x4 ⊕ x1x2 ⊕ x1x2x3
Generated Quantum Circuit:
                                                       
q_0: ───────────────────────────■────■────■────■────■──
                                │    │    │    │    │  
q_1: ────────────■────■────■────┼────┼────┼────■────■──
                 │    │    │    │    │    │    │    │  
q_2: ───────■────┼────┼────■────┼────■────■────┼────■──
            │    │    │    │    │    │    │    │    │  
q_3: ──■────┼────┼────■────■────■────┼────■────┼────┼──
     ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐
q_4: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘


Reversible quantum circuit with #inputs = #outputs

In [5]:
import itertools

def verify_reed_muller(truth_table, terms, n_vars, output_idx):
    """
    Verify if the given Reed-Muller expansion satisfies the truth table for a specific output.
    :param truth_table: Dictionary where keys are input binary strings and values are lists of outputs (0 or 1).
    :param terms: List of terms in the Reed-Muller expansion.
    :param n_vars: Number of variables in the function.
    :param output_idx: Index of the output to verify.
    :return: True if terms match the truth table for the given output, False otherwise.
    """
    for input_bits, outputs in truth_table.items():
        expected_output = outputs[output_idx]
        # Compute the function output for this input using the terms
        output = 0
        for term in terms:
            product = 1
            for i, bit in enumerate(term):
                if bit == '1':
                    product &= int(input_bits[i])
            output ^= product  # XOR the result
        if output != expected_output:
            return False
    return True

def find_reed_muller_expansion(truth_table, output_idx):
    """
    Find the Positive Polarity Reed-Muller (PPRM) expansion for a specific output in the truth table.
    :param truth_table: Dictionary where keys are input binary strings and values are lists of outputs (0 or 1).
    :param output_idx: Index of the output for which to find the PPRM expansion.
    :return: List of terms in the PPRM expansion.
    """
    n_vars = len(next(iter(truth_table.keys())))  # Number of variables
    all_terms = list(itertools.product(['0', '1'], repeat=n_vars))  # All possible terms
    valid_expansion = []

    # Iterate over all subsets of terms
    for subset_size in range(len(all_terms) + 1):
        for terms_subset in itertools.combinations(all_terms, subset_size):
            terms_subset = ["".join(term) for term in terms_subset]  # Convert tuples to strings
            if verify_reed_muller(truth_table, terms_subset, n_vars, output_idx):
                valid_expansion = terms_subset
                return valid_expansion  # Return the first valid expansion found

    return valid_expansion  # If no valid expansion is found

def pprm_to_expression(terms, n_vars):
    """
    Convert PPRM terms into a human-readable Boolean expression.
    :param terms: List of terms in PPRM (binary strings).
    :param n_vars: Number of input variables.
    :return: String representation of the PPRM expansion.
    """
    expression = []
    for term in terms:
        # Map binary to variable names
        variables = [f"x{i+1}" for i, bit in enumerate(term) if bit == '1']
        expression.append("".join(variables) if variables else "1")
    return " ⊕ ".join(expression)

# Example usage
truth_table = {
    '0000': [0, 1, 0, 1],
    '0001': [1, 0, 1, 0],
    '0010': [1, 1, 0, 0],
    '0011': [0, 0, 1, 1],
    '0100': [1, 1, 1, 0],
    '0101': [0, 0, 0, 1],
    '0110': [0, 1, 1, 0],
    '0111': [1, 0, 0, 1],
    '1000': [1, 0, 0, 1],
    '1001': [0, 1, 1, 0],
    '1010': [0, 0, 0, 1],
    '1011': [1, 1, 1, 0],
    '1100': [0, 0, 1, 1],
    '1101': [1, 1, 0, 0],
    '1110': [1, 0, 1, 0],
    '1111': [0, 1, 0, 1]
}

n_vars = len(next(iter(truth_table.keys())))
n_outputs = len(next(iter(truth_table.values())))

# Find PPRM expansions for all outputs
pprm_expressions = []
for output_idx in range(n_outputs):
    pprm_terms = find_reed_muller_expansion(truth_table, output_idx)
    pprm_expression = pprm_to_expression(pprm_terms, n_vars)
    pprm_expressions.append((pprm_terms, pprm_expression))

# Display PPRM expansions
for terms, expression in pprm_expressions:
    print(f"PPRM Terms: {terms}")
    print(f"PPRM Expression: {expression}")


PPRM Terms: ['0001', '0010', '0100', '1000']
PPRM Expression: x4 ⊕ x3 ⊕ x2 ⊕ x1
PPRM Terms: ['0000', '0001', '1000']
PPRM Expression: 1 ⊕ x4 ⊕ x1
PPRM Terms: ['0001', '0100']
PPRM Expression: x4 ⊕ x2
PPRM Terms: ['0000', '0001', '0010', '0100', '0110', '1010', '1100']
PPRM Expression: 1 ⊕ x4 ⊕ x3 ⊕ x2 ⊕ x2x3 ⊕ x1x3 ⊕ x1x2


In [6]:
pprm_expressions

[(['0001', '0010', '0100', '1000'], 'x4 ⊕ x3 ⊕ x2 ⊕ x1'),
 (['0000', '0001', '1000'], '1 ⊕ x4 ⊕ x1'),
 (['0001', '0100'], 'x4 ⊕ x2'),
 (['0000', '0001', '0010', '0100', '0110', '1010', '1100'],
  '1 ⊕ x4 ⊕ x3 ⊕ x2 ⊕ x2x3 ⊕ x1x3 ⊕ x1x2')]

We can thus obtain PPRM of any circuit given a truth table in the following form :

In [7]:
def findPPRM(pprm_expressions):
  ans = []
  for t,e in pprm_expressions:
    ans.append(t)
  return ans
findPPRM(pprm_expressions)

[['0001', '0010', '0100', '1000'],
 ['0000', '0001', '1000'],
 ['0001', '0100'],
 ['0000', '0001', '0010', '0100', '0110', '1010', '1100']]

In this form of PPRM : '000' maps to 1 and and the index where the term has value 1 means that literal is present . for e.g : [000,101] will mean 1 xor x0. x2  (as 1 present in index 0 and 2 of '101' and '000' means 1). For multiple outputs (here = #inputs = 3) the pprm will have 3 such lists for 3 outputs .

Lets consider the example given in paper and we obtain the following PPRM for :

ao = 1 ⊕ a

bo = c ⊕ b ⊕ ac

co = b ⊕ ac ⊕ ab

In [8]:
pprm = [
    ['000', '100'],
    ['001', '010', '101'],
    ['010', '101', '110']
]

In [9]:
initTerms = 0
for out in pprm:
  initTerms += len(out)
initTerms

8

In [10]:
def calculate_terms(pprm):
    """
    Calculate the number of terms in the PPRM expansion.
    :param pprm: Current PPRM expansion.
    :return: Number of terms.
    """
    initTerms = 0
    for out in pprm:
      initTerms += len(out)
    return initTerms

The following blocks are used to generate function for making the substitution : vi = vi ⊕ factor and obtain the PPRM

In [11]:
def interpret_pprm(pprm):
    """
    Converts a PPRM into Boolean expressions dynamically.

    Args:
    - pprm (list of list of str): PPRM representation, where each inner list corresponds to one output.

    Returns:
    - list of str: Boolean expressions for each output.
    """
    num_vars = len(pprm)  # Number of inputs/outputs (equal)
    variable_names = [f"x{i}" for i in range(num_vars)]  # Dynamically generate variable names

    expressions = []

    for terms in pprm:
        expr_terms = []  # Store terms for the current output

        for term in terms:
            if all(bit == '0' for bit in term):  # Special case: all '0's mean constant 1
                expr_terms.append("1")
            else:
                # Generate product (AND) terms based on binary encoding
                product = []
                for idx, bit in enumerate(term):
                    if bit == '1':  # Include the variable if the bit is 1
                        product.append(variable_names[idx])
                expr_terms.append(" & ".join(product))

        # Combine terms with XOR (⊕)
        expression = " ⊕ ".join(expr_terms)
        expressions.append(expression)

    return expressions


def substitute(pprm, vi, factor):
    """
    Substitutes a variable vi with a new term based on the given factor.

    Args:
    - pprm (list of list of str): PPRM representation.
    - vi (int): The variable index to substitute.
    - factor (str): The factor (like '000') that dictates how to modify the variable.

    Returns:
    - list of str: The modified Boolean expressions.
    """
    num_vars = len(factor)  # Number of inputs/variables
    variable_names = [f"x{i}" for i in range(num_vars)]  # Variable names: x0, x1, ...

    expressions = []
    ans = ''
    for f in range(len(factor)):
      if ans=='':
         if factor[f] == '1':
            ans += variable_names[f]
      else:
       if factor[f] == '1':
        ans += '&'+variable_names[f]

    if ans == '':
      ans = '1'
    for output_terms in pprm:
        expr_terms = []  # Store terms for this output

        for term in output_terms:
            if all(bit == '0' for bit in term):  # Special case: all '0's mean constant 1
                expr_terms.append("1")
            else:
                # For each term in the output, we will modify it based on the factor
                modified_term = []
                for idx, bit in enumerate(term):
                    if bit == '1':  # Check if the bit is 1
                        # If this is the variable to be substituted, modify it
                        if idx == vi:
                            modified_term.append(f"({ans} ⊕ {variable_names[idx]})")
                        else:
                            modified_term.append(variable_names[idx])
                    else:
                        continue  # Skip zero bits

                expr_terms.append(" & ".join(modified_term))

        # Combine terms with XOR
        expression = " ⊕ ".join(expr_terms)
        expressions.append(expression)

    return expressions


# Example usage
pprm = [
    ['000', '100'],         # Output 1
    ['001', '010', '101'],  # Output 2
    ['010', '101', '110']   # Output 3
]

# Generate Boolean expressions
boolean_expressions = interpret_pprm(pprm)
print("Original Boolean Expressions:")
for i, expr in enumerate(boolean_expressions):
    print(f"Output {i + 1}: {expr}")

# Substitute vi = 0, factor = '000' and modify the expressions
vi = 0  # Substitute for x0
factor = '000'  # Factor to modify (for example, all 0s)
modified_expressions = substitute(pprm, vi, factor)

print("\nModified Boolean Expressions:")
for i, expr in enumerate(modified_expressions):
    print(f"Output {i + 1}: {expr}")


Original Boolean Expressions:
Output 1: 1 ⊕ x0
Output 2: x2 ⊕ x1 ⊕ x0 & x2
Output 3: x1 ⊕ x0 & x2 ⊕ x0 & x1

Modified Boolean Expressions:
Output 1: 1 ⊕ (1 ⊕ x0)
Output 2: x2 ⊕ x1 ⊕ (1 ⊕ x0) & x2
Output 3: x1 ⊕ (1 ⊕ x0) & x2 ⊕ (1 ⊕ x0) & x1


In [12]:
modified_expressions

['1 ⊕ (1 ⊕ x0)',
 'x2 ⊕ x1 ⊕ (1 ⊕ x0) & x2',
 'x1 ⊕ (1 ⊕ x0) & x2 ⊕ (1 ⊕ x0) & x1']

In [13]:
import itertools

def verify_reed_muller(truth_table, terms, n_vars):
    """
    Verify if the given Reed-Muller expansion satisfies the truth table.
    :param truth_table: Dictionary where keys are input binary strings and values are outputs (0 or 1).
    :param terms: List of terms in the Reed-Muller expansion.
    :param n_vars: Number of variables in the function.
    :return: True if terms match the truth table, False otherwise.
    """
    for input_bits, expected_output in truth_table.items():
        # Compute the function output for this input using the terms
        output = 0
        for term in terms:
            product = 1
            for i, bit in enumerate(term):
                if bit == '1':
                    product &= int(input_bits[i])
            output ^= product  # XOR the result
        if output != expected_output:
            return False
    return True

def find_reed_muller_expansion(truth_table):
    """
    Find the Positive Polarity Reed-Muller (PPRM) expansion for a given truth table.
    :param truth_table: Dictionary where keys are input binary strings and values are outputs (0 or 1).
    :return: List of terms in the PPRM expansion.
    """
    n_vars = len(next(iter(truth_table.keys())))  # Number of variables
    all_terms = list(itertools.product(['0', '1'], repeat=n_vars))  # All possible terms
    valid_expansion = []

    # Iterate over all subsets of terms
    for subset_size in range(len(all_terms) + 1):
        for terms_subset in itertools.combinations(all_terms, subset_size):
            terms_subset = ["".join(term) for term in terms_subset]  # Convert tuples to strings
            if verify_reed_muller(truth_table, terms_subset, n_vars):
                valid_expansion = terms_subset
                return valid_expansion  # Return the first valid expansion found

    return valid_expansion  # If no valid expansion is found

def pprm_to_expression(terms, n_vars):
    """
    Convert PPRM terms into a human-readable Boolean expression.
    :param terms: List of terms in PPRM (binary strings).
    :param n_vars: Number of input variables.
    :return: String representation of the PPRM expansion.
    """
    expression = []
    for term in terms:
        # Map binary to variable names
        variables = [f"x{i+1}" for i, bit in enumerate(term) if bit == '1']
        expression.append("".join(variables) if variables else "1")
    return " ⊕ ".join(expression)

# Example usage
truth_table = {
    '0000': 0,
    '0001': 1,
    '0010': 1,
    '0011': 0,
    '0100': 1,
    '0101': 0,
    '0110': 0,
    '0111': 1,
    '1000': 1,
    '1001': 0,
    '1010': 0,
    '1011': 1,
    '1100': 0,
    '1101': 1,
    '1110': 1,
    '1111': 0
}

# Find PPRM expansion
pprm_terms = find_reed_muller_expansion(truth_table)

# Convert to expression for readability
n_vars = len(next(iter(truth_table.keys())))
pprm_expression = pprm_to_expression(pprm_terms, n_vars)

print("PPRM Terms:", pprm_terms)
print("PPRM Expression:", pprm_expression)

PPRM Terms: ['0001', '0010', '0100', '1000']
PPRM Expression: x4 ⊕ x3 ⊕ x2 ⊕ x1


In [14]:
import itertools

def evaluate_expression(expression, variables):
    """
    Evaluates a Boolean expression for a given set of variable values.

    Args:
    - expression (str): Boolean expression to evaluate.
    - variables (dict): Dictionary mapping variable names to values.

    Returns:
    - int: Result of the evaluated expression (0 or 1).
    """
    # Replace variable names with their corresponding values
    for var, value in variables.items():
        expression = expression.replace(var, str(value))

    # Replace XOR (⊕) with bitwise XOR (^), and AND (&) remains the same
    expression = expression.replace("⊕", "^")

    # Evaluate the expression safely
    return eval(expression)

def generate_truth_table(expressions, num_vars):
    """
    Generates the truth table for a set of Boolean expressions.

    Args:
    - expressions (list of str): Boolean expressions to evaluate.
    - num_vars (int): Number of variables.

    Returns:
    - list of dict: Truth table as a list of rows, where each row is a dictionary.
    """
    variable_names = [f"x{i}" for i in range(num_vars)]
    truth_table = []

    # Generate all possible combinations of input variables (0 or 1)
    for values in itertools.product([0, 1], repeat=num_vars):
        row = {var: val for var, val in zip(variable_names, values)}

        # Evaluate each expression and add the results to the row
        for i, expr in enumerate(expressions):
            row[f"Output {i + 1}"] = evaluate_expression(expr, row)

        truth_table.append(row)

    return truth_table

# Test Cases
modified_expressions = [
    "1 ⊕ (1 ⊕ x0)",
    "x2 ⊕ x1 ⊕ (1 ⊕ x0) & x2",
    "x1 ⊕ (1 ⊕ x0) & x2 ⊕ (1 ⊕ x0) & x1"
]

num_vars = len(modified_expressions )  # Number of input variables (x0, x1, x2)= num_outputs
# Generate truth table
truth_table = generate_truth_table(modified_expressions, num_vars)
# Display the truth table
import pandas as pd
df = pd.DataFrame(truth_table)
print(df)

   x0  x1  x2  Output 1  Output 2  Output 3
0   0   0   0         0         0         0
1   0   0   1         0         0         1
2   0   1   0         0         1         0
3   0   1   1         0         1         1
4   1   0   0         1         0         0
5   1   0   1         1         1         0
6   1   1   0         1         1         1
7   1   1   1         1         0         1


In [15]:
def transform_truth_table(truth_table, num_vars):
    """
    Transforms the truth table into the desired format.

    Args:
    - truth_table (list of dict): Truth table as a list of dictionaries.
    - num_vars (int): Number of variables.

    Returns:
    - dict: Transformed truth table where keys are binary input strings
            and values are lists of output values.
    """
    transformed_table = {}

    for row in truth_table:
        # Generate the binary key based on the input variables
        binary_key = ''.join(str(row[f"x{i}"]) for i in range(num_vars))

        # Collect all output values for the current row
        output_values = [row[f"Output {i + 1}"] for i in range(len(row) - num_vars)]

        # Store in the transformed table
        transformed_table[binary_key] = output_values

    return transformed_table


# Generate truth table using previous function
truth_table = generate_truth_table(modified_expressions, num_vars)

# Transform the truth table to the desired format
transformed_table = transform_truth_table(truth_table, num_vars)

# Display the transformed table
import pprint
pprint.pprint(transformed_table)


{'000': [0, 0, 0],
 '001': [0, 0, 1],
 '010': [0, 1, 0],
 '011': [0, 1, 1],
 '100': [1, 0, 0],
 '101': [1, 1, 0],
 '110': [1, 1, 1],
 '111': [1, 0, 1]}


In [16]:
import itertools

def evaluate_expression_to_binary(expressions, input_binary):
    """
    Evaluates the Boolean expressions for a given input binary string.

    Args:
    - expressions (list of str): Boolean expressions to evaluate.
    - input_binary (str): Input binary string (e.g., '000').

    Returns:
    - str: Concatenated binary output string (e.g., '100').
    """
    num_vars = len(input_binary)
    variable_names = [f"x{i}" for i in range(num_vars)]

    # Map binary string to variable values
    variables = {var: int(bit) for var, bit in zip(variable_names, input_binary)}

    # Evaluate each expression
    output_binary = ""
    for expr in expressions:
        output_binary += str(evaluate_expression(expr, variables))

    return output_binary


def generate_truth_table_as_dict(expressions, num_vars):
    """
    Generates the truth table as a dictionary mapping input binary strings to output binary strings.

    Args:
    - expressions (list of str): Boolean expressions to evaluate.
    - num_vars (int): Number of variables.

    Returns:
    - dict: Truth table mapping input binary strings to output binary strings.
    """
    truth_table = {}

    # Generate all possible combinations of input variables (0 or 1)
    for values in itertools.product([0, 1], repeat=num_vars):
        input_binary = "".join(map(str, values))  # Convert input tuple to binary string
        output_binary = evaluate_expression_to_binary(expressions, input_binary)
        truth_table[input_binary] = output_binary

    return truth_table

In [17]:
def split_truth_table_by_output(truth_table):
    """
    Splits the given truth table into three truth tables,
    each representing a specific bit of the output as an integer.


    :param truth_table: Dictionary where keys and values are binary strings.
    :return: Three dictionaries, each corresponding to one bit of the output.
    """
    # Initialize three new truth tables
    truth_table_bit_0 = {}
    truth_table_bit_1 = {}
    truth_table_bit_2 = {}


    for input_bits, output_bits in truth_table.items():
        # Extract individual bits and convert them to integers
        bit_0 = int(output_bits[0])  # Most significant bit
        bit_1 = int(output_bits[1])  # Middle bit
        bit_2 = int(output_bits[2])  # Least significant bit


        # Assign integer values to the corresponding truth tables
        truth_table_bit_0[input_bits] = bit_0
        truth_table_bit_1[input_bits] = bit_1
        truth_table_bit_2[input_bits] = bit_2


    return truth_table_bit_0, truth_table_bit_1, truth_table_bit_2




# Example usage
truth_table_dict = {
    '000': '100',
    '100': '000',
    '010': '111',
    '110': '010',
    '001': '110',
    '101': '001',
    '011': '101',
    '111': '011',
}


# bit_0_table, bit_1_table, bit_2_table = split_truth_table_by_output(truth_table)


# print("Bit 0 Table:", bit_0_table)
# print("Bit 1 Table:", bit_1_table)
# print("Bit 2 Table:", bit_2_table)


for table in split_truth_table_by_output(truth_table_dict):
    # Find PPRM expansion
    pprm_terms = find_reed_muller_expansion(table)


    # Convert to expression for readability
    n_vars = len(next(iter(table.keys())))
    pprm_expression = pprm_to_expression(pprm_terms, n_vars)


    print("PPRM Terms:", pprm_terms)
    print("PPRM Expression:", pprm_expression)

PPRM Terms: ['000', '100']
PPRM Expression: 1 ⊕ x1
PPRM Terms: ['001', '010', '101']
PPRM Expression: x3 ⊕ x2 ⊕ x1x3
PPRM Terms: ['010', '101', '110']
PPRM Expression: x2 ⊕ x1x3 ⊕ x1x2


In [18]:
def substitutefact(pprm,vi,factor):
  num_vars = len(pprm)  # Number of input variables (x0, x1, x2)
  boolean_expressions = interpret_pprm(pprm)
  modified_expressions = substitute(pprm, vi, factor)
  # print(f'modified : {modified_expressions}')
  truth_table = generate_truth_table(modified_expressions, num_vars)
  # print(f'truth_table : {truth_table}')
  # Generate truth table using previous function
  # Transform the truth table to the desired format
  transformed_table = transform_truth_table(truth_table, num_vars)
  truth_table_dict = generate_truth_table_as_dict(modified_expressions, num_vars)

  truth_table = truth_table_dict

  pprm_expressions=[]
  for table in split_truth_table_by_output(truth_table):
      # Find PPRM expansion
      pprm_terms = find_reed_muller_expansion(table)


      # Convert to expression for readability
      # n_vars = len(next(iter(table.keys())))
      # pprm_expression = pprm_to_expression(pprm_terms, n_vars)
      pprm_expressions.append(pprm_terms)
  return pprm_expressions

In [19]:
pprm = [
    ['000', '100'],
    ['001', '010', '101'],
    ['010', '101', '110']
]

In [20]:
p=substitutefact(pprm,0,'000')
print(p)

[['100'], ['010', '101'], ['001', '101', '110']]


In [29]:
import heapq

def calculate_terms(pprm):
    """
    Calculate the number of terms in the PPRM expansion.
    :param pprm: Current PPRM expansion.
    :return: Number of terms.
    """
    initTerms = 0
    for out in pprm:
      initTerms += len(out)
    return initTerms

pprm = [
    ['000', '100'],
    ['001', '010', '101'],
    ['010', '101', '110']
]
num_vars=len(pprm)
initTerms = calculate_terms(pprm)

def calculate_priority(node, alpha=0.3, beta=0.6, gamma=0.1):
    """
    Calculate the priority of a node based on the provided weights.
    :param node: Node object.
    :param alpha, beta, gamma: Weights for terms, depth, and factor literal count.
    """
    return (alpha * node.depth) + (beta * node.elim / node.depth) -  gamma * node.factor[1].count('1')




def vi_string(vi):
  ans = ''
  for i in range(num_vars):
    if i==vi:
      ans+='1'
    else:
      ans+='0'
  return ans




def is_solution(node):
    """
    Check if a given node represents a valid solution.
    :param node: Current node in the search tree.
    :return: True if the node is a solution, False otherwise.
    """
    for v in range(len(node.pprm)):
        if len(node.pprm[v]) != 1:
            # print(f'node.pprm[v]={node.pprm[v]}')
            # print(f'node.pprm[v][0][v]={node.pprm[v][0][v]}')
            return False
        if not (node.pprm[v][0][v] == '1' and all(node.pprm[v][0][i] == '0' for i in range(len(node.pprm[v][0])) if i != v)):
            return False
    return True





In the best solution node we should  obtain the Synthesized Network: [['100'], ['010'], ['001']] which means a_out = a , b_out= b , c_out= c exactly the condition required to get reversible circuit .  Now to track correctly the transformations , in node class we define an object transformation which stores the list of transformations done to get the solution


In [30]:
class Node:
    def __init__(self, parent=None, pprm=pprm, depth=0, factor=None, terms=0, priority=float('inf'), transformations=None):
        self.parent = parent
        self.pprm = pprm  # PPRM expansion at this node
        self.depth = depth
        self.factor = factor  # Factor associated with this node
        self.terms = calculate_terms(self.pprm)  # Number of terms in the PPRM expansion
        self.elim = initTerms - self.terms  # Number of eliminated terms
        self.priority = priority  # Priority for the priority queue
        # Track the sequence of transformations
        self.transformations = transformations if transformations is not None else []

    def __lt__(self, other):
        return self.priority < other.priority  # For priority queue sorting

def synthesize_pprm(pprm, max_time):

    best_depth = float('inf')
    best_sol_node = None

    # Initialize the root node of the search tree
    root_node = Node(pprm=pprm, depth=0, terms=initTerms)

    # Initialize priority queue and push the root node
    priority_queue = []
    heapq.heappush(priority_queue, (-root_node.priority, root_node))

    while priority_queue:
        _, parent_node = heapq.heappop(priority_queue)

        if parent_node.depth >= best_depth - 1:
            continue

        for vi in range(len(parent_node.pprm)):  # Loop over output variables in PPRM
            vi_pprm = parent_node.pprm[vi]  # Expansion for the current output variable

            for factor in vi_pprm:
                if factor[vi] == '0':  # Skip factors that don't contain vi
                    # Create a child node
                    new_transformations = parent_node.transformations + [[vi_string(vi), factor]]
                    child_node = Node(
                        parent=parent_node,
                        depth=parent_node.depth + 1,
                        factor=[vi_string(vi), factor],
                        transformations=new_transformations
                    )

                    # Substitute vi ⊕ factor in all PPRM expansions of output
                    child_node.pprm = substitutefact(parent_node.pprm, vi, factor)
                    child_node.terms = calculate_terms(child_node.pprm)
                    child_node.elim = parent_node.terms - child_node.terms

                    # If a solution is found, update the best solution
                    if is_solution(child_node) and child_node.depth < best_depth:
                        best_depth = child_node.depth
                        best_sol_node = child_node

                    # Calculate priority and push child node if it's a candidate for further exploration
                    if child_node.elim > 0:
                        child_node.priority = calculate_priority(child_node)
                        heapq.heappush(priority_queue, (-child_node.priority, child_node))

        max_time -= 1  # Decrement the timer for each iteration

    # Reconstruct the synthesized network from the best solution
    return best_sol_node.pprm, best_sol_node.transformations

In [31]:
bestnode,transformation = synthesize_pprm(pprm, 100)

In [32]:
bestnode

[['100'], ['010'], ['001']]

In [33]:
transformation

[['100', '000'], ['010', '101'], ['001', '110']]

Finally , generate quantum circuit from this list of transformations

In [28]:
from qiskit import QuantumCircuit


def construct_quantum_circuit(transformations):
    """
    Construct a quantum circuit based on the transformations:
    1. a = a ^ 1
    2. b = b ^ (a AND c)
    3. c = c ^ (a AND b)

    :return: QuantumCircuit object
    """
    num_qubits = num_vars  # Number of qubits (3 in this case)
    # Initialize a quantum circuit with 3 qubits
    qc = QuantumCircuit(num_qubits)

    for t in transformation :

      for i in range(len(t[0])):
        if t[0][i]=='1':
          target = i
      if t[1] == '0'*num_qubits:
        qc.x(target)
      else:
        controls = []
        for i in range(len(t[1])):
          if t[1][i]=='1':
            controls.append(i)
        qc.mcx(controls,target)

    return qc





# Generate the circuit
quantum_circuit = construct_quantum_circuit(transformation)

# Display the quantum circuit
print("Generated Quantum Circuit:")
print(quantum_circuit.draw('text'))

Generated Quantum Circuit:
     ┌───┐          
q_0: ┤ X ├──■────■──
     └───┘┌─┴─┐  │  
q_1: ─────┤ X ├──■──
          └─┬─┘┌─┴─┐
q_2: ───────■──┤ X ├
               └───┘


Succesfully generated the Quantum Circuit !!